In [1]:
from bs4 import BeautifulSoup
import requests
import csv

In [88]:
search_state = input('Enter Search State (NO CAPS!!!): ')

Enter Search State (NO CAPS): florida


In [91]:
url = "https://health.usnews.com/doctors/"+search_state
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36 OPR/106.0.0.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
page = requests.get(url,headers=headers)
code = BeautifulSoup(page.content, "html.parser")

In [93]:
#this is just something i do when there's a mistake to be able to see what python sees when it scrapes
#sometimes it's different from what I am looking at in the browser for one reason or another
#with open('page.html', 'w', encoding='UTF8') as f:
#    f.write(str(code))

In [94]:
max_page = code.find_all('a', class_='page-numbers__Button-sc-138ov1k-4 biGwcb')
max_page2 = max_page[-1].text.strip()
pages = []
for i in range(int(max_page2)):
    pages.append('https://health.usnews.com/doctors/florida?&page_num='+str(i+1))

In [95]:
doctor_links = []
for x in range(len(pages)):
    url2 = pages[x]
    page2 = requests.get(url2,headers=headers)
    code2 = BeautifulSoup(page2.content, "html.parser")
    links2 = []
    links = code2.find_all('a', href=True)
    links = [x['href'] for x in links]
    for i in range(len(links)):
        if links[i][:8]=='/doctors':
            links2.append('https://health.usnews.com'+links[i])
    links2 = links2[2:-5]
    links2 = list(set(links2))
    doctor_links.extend(links2)


In [ ]:
for i in doctor_links:
    row = []
    url3 = i
    page3 = requests.get(url3,headers=headers)
    code3 = BeautifulSoup(page3.content, "html.parser")
    
    name = code3.find('h1', class_='Heading-sc-1w5xk2o-0 Hero__Name-sc-1lw4wit-4 jkCClc dRlwZH').text.strip()
    if name[:4] == 'Dr. ':
        name = name[4:len(name)-3]
    else:
        name = name[:len(name)-3]
    #name
    row.append(name)

    specialty = ''
    try:
        specialty = code3.find('h3', class_='Heading-sc-1w5xk2o-0 Specialties__SpecialtyName-mzebq4-4 eceUqc dCoRLC').text.strip()
    except:
        pass
    row.append(specialty)

    subspecialty = code3.find_all('h3', class_='Heading-sc-1w5xk2o-0 Specialties__Subspecialty-mzebq4-3 gBclBW dheGEC')
    subspecialty = [y.text.strip() for y in subspecialty]
    subspecialty = ', '.join(subspecialty)
    row.append(subspecialty)

    languages = code3.find('div', class_='content__InfoItem-sc-1g5jtse-9 hMmtt').text.strip()
    languages = languages[7:]
    row.append(languages)

    hospitals = code3.find_all('div', class_='Box-w0dun1-0 Legend__Key-mo6leh-4 eoGdsS cqVtTh')
    hospitals = [y.text.strip()[1:] for y in hospitals]
    hospitals2 = ', '.join(hospitals)
    row.append(hospitals2)

    pat_awards = code3.find_all('p', class_='Paragraph-sc-1iyax29-0 IconList__ParagraphReversed-emyavm-4 eXlyGl IconList__Headline-emyavm-5 headline')
    pat_awards = [y.text.strip() for y in pat_awards]
    pat_awards2 = ', '.join(pat_awards)
    row.append(pat_awards2)

    remark = ''
    stars = ''
    try:
        remark = code3.find('h4', class_='Heading-sc-1w5xk2o-0 hDrxiz').text.strip()
        stars = code3.find('p', class_='Paragraph-sc-1iyax29-0 fHZMjV ml3').text.strip()
    except:
        pass
    stars = stars[1:4]
    if stars != '':
        pat_exp = remark + ', ' + stars + ' stars'
    else:
        pat_exp = ''
    row.append(pat_exp)

    tot_reviews = ''
    try:
        tot_reviews = code3.find('p', class_='Paragraph-sc-1iyax29-0 ftPWus').text.strip()
        tot_reviews = tot_reviews[:tot_reviews.index(' ')]
    except:
        pass
    row.append(tot_reviews)

    feedbacks = code3.find_all('h4', class_='Heading-sc-1w5xk2o-0 TableScorecard__ColumnTitle-sc-1azu7ro-8 RmATh TvRUG')
    feedbacks = [y.text.strip() for y in feedbacks]
    notes = code3.find_all('p', class_='Paragraph-sc-1iyax29-0 TableScorecard__RatingLabel-sc-1azu7ro-7 jUIsdH iZuiEt')
    notes = [y.text.strip() for y in notes]
    f_stars = code3.find_all('p', class_='Paragraph-sc-1iyax29-0 dkGWLZ ml2')
    f_stars = [y.text.strip() for y in f_stars]
    f_stars = [y[1:4] + ' stars' for y in f_stars]
    feedbacks2 = []
    for i in range(len(feedbacks)):
        feedbacks2.append(feedbacks[i]+' - '+notes[i]+', '+f_stars[i])
    feedbacks3 = ', '.join(feedbacks2)
    row.append(feedbacks3)

    address = code3.find_all('p', class_='Paragraph-sc-1iyax29-0 dDlwaN')[-1].text.strip()
    row.append(address)

    phone = ''
    try:
        phone = code3.find('a', class_='Anchor-byh49a-0 lhoIMZ').text.strip()
        if phone[0]=='1':
            phone = '('+phone[2:5]+') '+phone[6:]
        else:
            pass
    except:
        pass
    row.append(phone)

    corporate = ''
    try:
        corporate = code3.find('h4', class_='Heading-sc-1w5xk2o-0 content__PracticeName-sc-1g5jtse-7 eBwjvO hQyHAm').text.strip()
    except:
        pass
    row.append(corporate)

    qualifications = code3.find_all('p', class_='Paragraph-sc-1iyax29-0 cvyYGJ')
    qualifications = [y.text.strip() for y in qualifications]

    qualifications2 = code3.find_all('p', class_='Paragraph-sc-1iyax29-0 euRmcR')
    qualifications2 = [y.text.strip() for y in qualifications2]

    med_school = []
    certifications = []
    licences = []
    internship = []
    residency = []
    fellowship = []
    for i in range(len(qualifications2)):
        if qualifications2[i] == 'Medical School':
            med_school.append(qualifications[i])
        if qualifications2[i][:9] == 'Residency':
            residency.append(qualifications[i]+qualifications2[i][9:])
        if qualifications2[i][:10] == 'Internship':
            internship.append(qualifications[i]+qualifications2[i][10:])
        if qualifications2[i][:10] == 'Fellowship':
            fellowship.append(qualifications[i]+qualifications2[i][10:])
        if qualifications2[i][:9] == 'Certified':
            certifications.append(qualifications[i]+' - '+qualifications2[i])    
        if qualifications[i][-7:] == 'License':
            licences.append(qualifications[i]+' - '+qualifications2[i])

    fellowship2 = ', '.join(fellowship)
    internship2 = ', '.join(internship)
    residency2 = ', '.join(residency)
    med_school2 = ', '.join(med_school)
    certifications2 = ', '.join(certifications)
    licences2 = ', '.join(licences)
    row.extend([med_school2,internship2,residency2,fellowship2,certifications2,licences2])

    overview = code3.find('div', class_='Raw-slyvem-0 eSouqg').text.strip()
    row.append(overview)

    spec_details = ''
    try:
        spec_details = code3.find('div', class_='Raw-slyvem-0 Specialties__Description-mzebq4-2 bWVoAf fsgwfx').text.strip()
    except:
        pass
    row.append(spec_details)

    with open(search_state+"_doctors_database.csv", 'a', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(row)